Muneel Haider

i21-0640

Assignment 1 - NLP

In [6]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import string
import random

nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Muneel
[nltk_data]     Haider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def readCSV(file):
    print("Loading csv file.")
    return pd.read_csv(file)

def cleanAndTokenize(textRead):
    textRead = textRead.lower()
    textRead = textRead.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(textRead)
    return tokens

In [8]:
def buildNGramModel(tokensList, n=1):
    print(f"Building {n}-gram model...")
    model = defaultdict(Counter)
    for tokens in tokensList:
        for i in range(len(tokens) - (n - 1)):
            modelDictionary = tuple(tokens[i:i + n - 1])
            model[modelDictionary][tokens[i + n - 1]] += 1
    return model

In [9]:
def predictNextWord(model, modelDictionary):
    modelDictionary = tuple(modelDictionary)
    if modelDictionary in model:
        return max(model[modelDictionary], key=model[modelDictionary].get)
    else:
        return None

def predictFrequentWord(model):
    return max(model[()], key=model[()].get)

def generateSentence(model, modelDictionary, countWords=10):
    sentence = list(modelDictionary)
    for _ in range(countWords - len(modelDictionary)):
        nextWord = predictNextWord(model, modelDictionary)
        if not nextWord:
            break
        sentence.append(nextWord)
        modelDictionary = tuple(sentence[-len(modelDictionary):])
    return ' '.join(sentence)

In [10]:
def reviewClassifier(reviewTokens, model):
    print("Classifying review...")
    positiveScore = sum(model[()][token] for token in reviewTokens if token in model[()])
    negativeScore = sum(-model[()][token] for token in reviewTokens if token not in model[()])
    return "Positive" if positiveScore > negativeScore else "Negative"

In [11]:
file = 'IMDB Dataset.csv'
data = readCSV(file)
data['tokens'] = data['review'].apply(cleanAndTokenize)

Unigram = buildNGramModel(data['tokens'], 1)
Bigram = buildNGramModel(data['tokens'], 2)
Trigram = buildNGramModel(data['tokens'], 3)

unigramSentence = generateSentence(Unigram, (predictFrequentWord(Unigram),), 10)
bigramSentence = generateSentence(Bigram, (unigramSentence.split()[0],), 10)
trigramSentence = generateSentence(Trigram, tuple(bigramSentence.split()[:2]), 10)

print("Generated Unigram Sentence:", unigramSentence)
print()
print("Generated Bigram Sentence:", bigramSentence)
print()
print("Generated Trigram Sentence:", trigramSentence)
print()

print("Unigram Sentence Classification:", reviewClassifier(unigramSentence.split(), Unigram))
print()
print("Bigram Sentence Classification:", reviewClassifier(bigramSentence.split(), Unigram))
print()
print("Trigram Sentence Classification:", reviewClassifier(trigramSentence.split(), Unigram))
print()

Loading csv file.
Building 1-gram model...
Building 2-gram model...
Building 3-gram model...
Generated Unigram Sentence: the

Generated Bigram Sentence: the film is a lot of the film is a

Generated Trigram Sentence: the film is a very good and the film is

Classifying review...
Unigram Sentence Classification: Positive

Classifying review...
Bigram Sentence Classification: Positive

Classifying review...
Trigram Sentence Classification: Positive

